<p align=center> <img src='https://resource.calcionapoli24.it/www/galleria/upload/1563696286-netflixlogo.jpg' width='700'> </p>

<h1 align='center'> SEMPLE NETFLIX RECCOMENDATION SYSTEM </h1>

<h3 align=left>
    <br> - PHASE 1 --> Preparation </br>
    <br> - PHASE 2 --> Exploratory Data Analysis - EDA </br>
</h3>

In [6]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import requests
from sklearn.preprocessing import MultiLabelBinarizer
import re
from wordcloud import WordCloud, STOPWORDS
#%matplotlib inline
print("Libraries have been correctly imported!")

Libraries have been correctly imported!


In [7]:
#Nella scrittura del filepath ruotiamo sempre gli slash. Non dobbiamo mai avere backslash \ ma solo slash /
netflix_dataset='https://raw.githubusercontent.com/FrancescoCrovella/Netflix_EDA_kMEANS/main/netflix_titles.csv'
netflix_df=pd.read_csv(netflix_dataset)

print("The dataset has been correctly imported!")

netflix_df.head(2)

The dataset has been correctly imported!


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."


In [8]:
netflix_df.dtypes

show_id         object
type            object
title           object
director        object
cast            object
country         object
date_added      object
release_year     int64
rating          object
duration        object
listed_in       object
description     object
dtype: object

<h1 align=center> PHASE 1/2 </h3>
<h1 align=center> DATA CLEANING & PREPROCESSING </h3>

##### These are the most important basic information methods for a dataset:
- df.shape ---> number of rows and columns
- df.columns ---> columns array
- df.count() ---> number of records
- df.info() ---> different file info, including dtypes
- df.drop_duplicates()
- df.describe()
- df.nunique()


- GESTIONE COLONNE NON NECESSARIE
- df.drop(["columnName"],axis=1,inplace=True)


- GESTIONE MISSING VALUES
- df.isna().sum(axis=0) --- calcolo delle celle nulle per ogni colonna


- GESTIONE FORMATO DATA

In [9]:
netflix_df.shape

(8807, 12)

In [10]:
netflix_df.columns

Index(['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'duration', 'listed_in', 'description'],
      dtype='object')

In [11]:
netflix_df.count()

show_id         8807
type            8807
title           8807
director        6173
cast            7982
country         7976
date_added      8797
release_year    8807
rating          8803
duration        8804
listed_in       8807
description     8807
dtype: int64

In [12]:
#si capisce che alcune colonne hanno delle celle vuote. Capiamo qual è l'index massimo

In [13]:
netflix_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


In [14]:
#poichè l'index massimo è di 8807 dobbiamo capire cosa fare con le celle vuote

In [15]:
#facciamo droppare eventuali duplicati
netflix_df.drop_duplicates()
print('duplicates dropped')

duplicates dropped


In [16]:
netflix_df.nunique()

show_id         8807
type               2
title           8807
director        4528
cast            7692
country          748
date_added      1767
release_year      74
rating            17
duration         220
listed_in        514
description     8775
dtype: int64

In [17]:
#we need to cast date_added column since noy is an object type and should be a datatime
#pandas uses this format yyyy-mm-dd
netflix_df[['date_added']]=netflix_df[['date_added']].apply(pd.to_datetime)
netflix_df.head(2)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,2021-09-25,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,2021-09-24,2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."


In [18]:
print(netflix_df['date_added'].dtype)

datetime64[ns]


In [19]:
netflix_df.describe()

,release_year
count,8807.000000
mean,2014.180198
std,8.819312
min,1925.000000
25%,2013.000000
50%,2017.000000
75%,2019.000000
max,2021.000000


In [20]:
#Capiamo se ci sono colonne inutili da eliminare
netflix_df.drop(['show_id'],axis=1,inplace=True)

In [21]:
#capiamo se ci sono celle nulle per ogni colonna
netflix_df.isna().sum(axis=0)

type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64

### capiamo come gestire le celle nulle
### visto che negli show non ci sono celle nulle, allora non possiamo eliminare nessuna riga ma dobbiamo fare delle sostituzioni

- colonna director   --------> filliamo con "Unknown"
- colonna cast       --------> filliamo con "Unknown"
- colonna country    --------> filliamo con "Unknown"
- colonna date_added --------> inseriamo la DUMMY DATE ---> 1 gennaio 1900
- colonna rating     --------> filliamo con "Unknown"
- colonna duration   --------> filliamo con "Unknown"

In [22]:
empty_columns_tobefilled_unknown=['director','cast','country','rating','duration']

for i in netflix_df[empty_columns_tobefilled_unknown]:
    netflix_df[i]= netflix_df[i].fillna('Unknown')

netflix_df['date_added'].fillna(pd.Timestamp('1900-01-01'), inplace=True)

#controlliamo che se ricalcoliamo i valori nulli compaia tutto zero
netflix_df.isna().sum(axis=0)

type            0
title           0
director        0
cast            0
country         0
date_added      0
release_year    0
rating          0
duration        0
listed_in       0
description     0
dtype: int64

In [23]:
netflix_df

,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,Movie,Dick Johnson Is Dead,Kirsten Johnson,Unknown,United States,2021-09-25,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,TV Show,Blood & Water,Unknown,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,2021-09-24,2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",Unknown,2021-09-24,2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,TV Show,Jailbirds New Orleans,Unknown,Unknown,Unknown,2021-09-24,2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,TV Show,Kota Factory,Unknown,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,2021-09-24,2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
...,...,...,...,...,...,...,...,...,...,...,...
8802,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,2019-11-20,2007,R,158 min,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a..."
8803,TV Show,Zombie Dumb,Unknown,Unknown,Unknown,2019-07-01,2018,TV-Y7,2 Seasons,"Kids' TV, Korean TV Shows, TV Comedies","While living alone in a spooky town, a young g..."
8804,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,2019-11-01,2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...
8805,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,2020-01-11,2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero..."


In [24]:
netflix_df.head(2)

,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,Movie,Dick Johnson Is Dead,Kirsten Johnson,Unknown,United States,2021-09-25,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,TV Show,Blood & Water,Unknown,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,2021-09-24,2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."


# SISTEMA DI RACCOMANDAZIONE BASATO SUI LIKE E SULLA CORRELAZIONE DI
- TYPE
- COUNTRY
- LISTED_IN / CATEGORY

### COSTRUZIONE DI UNA FUNZIONE CHE RESTITUISCE UN ELENCO DI RACCOMANDAZIONI SULLA BASE DI UN LIKE ARBITRARIO

In [25]:
netflix_df.head(2)

,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,Movie,Dick Johnson Is Dead,Kirsten Johnson,Unknown,United States,2021-09-25,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,TV Show,Blood & Water,Unknown,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,2021-09-24,2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."


In [26]:
netflix_df.set_index('title',inplace=True)

In [27]:
def reccomendation(like):

    try:
        type=netflix_df.loc[like,'type']
        country=netflix_df.loc[like,'country']
        genere=netflix_df.loc[like,'listed_in']
        #rating=netflix_df.loc[like,'rating']

        request_1=netflix_df[netflix_df['country']==country]
        request_2=request_1[request_1['listed_in']==genere]
        request_3=request_2[request_2['type']==type]
        #request_4=request_3[request_3['rating']==rating]

        reccomendation_list_complete=(request_3.index.tolist())

        list_number_ofElements=len(reccomendation_list_complete)-1

        if list_number_ofElements >0:
            reccomendation_list=(reccomendation_list_complete[0:5])
            reccomendation_list.remove(like)
            reccomendation_list_toBullet=('\n\u2022'.join(reccomendation_list))
            reccomendation_comment=('\nNoi ti consigliamo di guardare anche: \n')
            print(reccomendation_comment + '\n\u2022' + reccomendation_list_toBullet)
        else:
            print('\nCi dispiace ma nella nostra lista non abbiamo nulla da consigliarti per il momento!')

    except:
        print('\nCi dispiace ma nel nostro catalogo non è presente questo contenuto!')


In [28]:
like= str(input("Inserisci una serie TV o film che hai già visto: "))

OurReccomendation=reccomendation(like)

#alternative method instead of interactive input
#like='Grey\'s Anatomy'


Noi ti consigliamo di guardare anche: 

•Cheers
•Frasier
•The Andy Griffith Show
